<h1>Line Following Algorithm</h1>
<h2>Step 1: Colour Identification</h2>
<p>The first step is to define the functions that take an image and convert it into the output image of the image processing pipeline.</p>

In [10]:
import cv2
import numpy as np

# Convert yellow in image
def getYellow(color):
    blurred = cv2.GaussianBlur(color, (3, 3), 0)

    # Convert to HSV
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, (15, 25, 25), (50, 255,255))

    # Slice the Yellow
    imask = mask>0
    yellow = np.zeros_like(blurred, np.uint8)
    
    # Apply mask
    yellow[imask] = blurred[imask]

    gray = cv2.cvtColor(yellow, cv2.COLOR_BGR2GRAY) 
    
    # Apply binary thershold
    (T, thresh) = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY) 

    # Remove ignored ignore regions
    thresh[:40,:]=0 #top

    return thresh

# Convert blue in image
def getBlue(color):
    blurred = cv2.GaussianBlur(color, (3, 3), 0)

    # Convert to HSV
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, (90, 80, 25), (105, 255,255))
    
    # Slice the Blue
    imask = mask>0
    blue = np.zeros_like(blurred, np.uint8)
    
    # Apply the mask
    blue[imask] = blurred[imask]

    gray = cv2.cvtColor(blue, cv2.COLOR_BGR2GRAY)
    
    #cApply binary threshold
    (T, thresh) = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    
    # Remove ignored ignore regions
    thresh[:40,:]=0 #top
    
    return thresh

# Get the number of white pixels in the image
def getWhite(image):
    return np.sum(image == 255)


<h2>Step 2: Define and Initailise Camera Classs </h2>
The next step is to define the camera class and the capture frames method which is also used to get the processed image

In [11]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv 
import cv2
import numpy as np

#using realsense to capture the color image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()
        self.flag = 0
        self.count = 0
        self.line_color = 0
        self.predict = [0,0,0,0,0,0,0,0,0]
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        
    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 
            
            # Identitfy color of rope in the current frame and update poll
            if getWhite(getYellow(image)) > getWhite(getBlue(image)):
                self.predict = [1]+self.predict[:-1]
            else: self.predict = [-1]+self.predict[:-1]
            
            # Check poll to find the current rope colour
            if sum(self.predict) >= 0:
                self.line_color = 0
            if sum(self.predict) < 0:
                self.line_color = 1
           
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

RuntimeError: failed to set power state

<h2>Step 3: Define the Process Update Function</h2>
<p>Start the camera and dispay the processed image. Then analyse the image to identify what action the robot should perform. <p>

In [3]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys
import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()



#create widgets for the displaying of the image
display_image = widgets.Image(format='jpeg', width='45%') #determine the width of the YELLOW image
#display_blue = widgets.Image(format='jpeg', width='45%')  #determine the width of the BLUE image
layout=widgets.Layout(width='100%')

#sidebyside = widgets.HBox([display_yellow, display_blue],layout=layout) #horizontal 
display(display_image) #display the widget



#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    image = change['new'] #retrieve data from the input dict
    
    # Display the image
    if camera.line_color == 0:
        display_image.value = bgr8_to_jpeg(getYellow(cv2.resize(image,(160,120))))
        output_image = getYellow(cv2.resize(image,(160,120)))
    if camera.line_color == 1:
        display_image.value = bgr8_to_jpeg(getBlue(cv2.resize(image,(160,120))))
        output_image = getBlue(cv2.resize(image,(160,120)))
    
    # Look for the amount of white in the image - ie the amount of path that the robot can see
    if getWhite(output_image[:,60:100])>60:
        camera.flag = 1
    elif getWhite(output_image[:,:40])>60:
        # Turn left
        camera.flag = 2
    elif getWhite(output_image[:,120:])>60:
        # Turn right
        camera.flag = 3



camera.observe(process, names='color_value')


Image(value=b'', format='jpeg', width='45%')

<h2>Step 4: Create the robot movement loop</h2>

In [14]:
import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

#this for loop will probably runs for 10 seconds
for i in range(800):
    if camera.flag == 1:              # Forward flag
        robot.forward(1) 
    elif camera.flag == 2:             # Left flag
        robot.left(0.4)
    elif camera.flag == 3:             # right flag
        robot.right(0.4)
    time.sleep(0.05)

        
#stop the robot
robot.stop()